# GEOtiled Computation Times using SAGA with Various Tile Counts

The goal of this notebook is to evaluate compute times of the slope terrain parameter using SAGA API and splitting the elevation data into a various number of tiles. The theory is that computation time will have a bell-shaped parabolic behavior as the number of tiles elevation is split into increases.

Note that what is being timed is the following:
* Conversion of elevation tiles into the SAGA format
* Computation of slope on each SAGA elevation file
* Conversion of computed slope tiles back into GeoTIFF
* Mosaic of the tiles into a single image

A base case is included under the 'Entire Image' section where the original elevation is not split into tiles.

## Initial Setup

In [1]:
# DO NOT MODIFY: Import libraries used to run notebook
import geotiled
import time
import os

/usr/lib/grass78/etc/python/grass/script/db.py:125: SyntaxWarning: invalid escape sequence '\g'
  """Perform SQL select statement


In [3]:
# Set working directory
data_folder = "/media/volume/geotiled-saga/tn_10m" 
geotiled.set_working_directory(data_folder)

## Optional: Output Codes

In [7]:
geotiled.print_data_codes()

The data codes and their associated parameter are:
30m : National Elevation Dataset (NED) 1 arc-second Current
10m : National Elevation Dataset (NED) 1/3 arc-second Current


In [ ]:
geotiled.print_parameter_codes()

In [ ]:
geotiled.print_region_codes()

## Configure Parameters

In [4]:
shapefile = "TN" # Region to cover
dataset = "10m"  # Dataset to download
params = ["SLP"] # Parameters to compute

## Download Data

In [5]:
# Set working directory
#data_folder = "/media/volume/gabriel-geotiled-small/tn_10m" 
#geotiled.set_working_directory(data_folder)

# Create a text file with download URLs from a shapefile
geotiled.fetch_dem(shapefile=shapefile, dataset=dataset, save_to_txt=False, download_folder="dem_tiles", download=True)

# Build mosaic from DEMs
geotiled.build_mosaic(input_folder="dem_tiles", output_file="mosaic", description="Elevation", cleanup=True)

# Reproject the mosaic to Projected Coordinate System (PCS) EPSG:9822 - Albers Conic Equal Area projection 
geotiled.reproject(input_file="mosaic", output_file="elevation", projection="EPSG:9822", cleanup=True)

Reading in shape file...


Downloading: 100%|█████████████████████████████████████████████| 58.1M/58.1M [00:02<00:00, 25.4MB/s]


Setting boundary extents...
Requesting data from USGS...


Downloading: 100%|██████████████████████████████████████████████| 14.2G/14.2G [02:12<00:00, 107MB/s]


Fetch process complete.
Getting input files...
Constructing VRT...
Updating band description...
Cleaning intermediary files...
Mosaic process complete.
Reprojecting mosaic.tif...
Cleaning intermediary files...
Reprojection complete.


## Entire Image

#### Results
* TN 30m (1.7 GB) - 239
* TN 10m (15 GB) - 2482

In [9]:
# Set some paths
elev = os.path.join(data_folder,"elevation.tif")
slope = os.path.join(data_folder,"slope.tif")
saga_elev = os.path.join(data_folder,"elevation.sdat")
saga_slope = os.path.join(data_folder,"slope.sdat")

# Start timer
start_time = time.time()

# Convert files to SAGA format
geotiled.__geotiff_to_sdat(elev, saga_elev)

# Run SAGA command to compute slope
cmd = ["tmux", "new-session", "-d", "-s", "computeOneSession", "saga_cmd", "ta_morphometry", "0", "-ELEVATION", os.path.join(data_folder,"elevation.sgrd"), "-SLOPE", os.path.join(data_folder,"slope.sgrd")]
geotiled.__bash(cmd)
geotiled.__wait_on_tmux()

# Convert output file back to GeoTIFF
geotiled.__sdat_to_geotiff(saga_slope, slope)

# Output total computation time
print("Compute time for entire image: %s seconds" % (time.time() - start_time))

Compute time for entire image: 2482.4793417453766 seconds


## 4 Tile Run

#### Results
* TN 30m (1.7 GB) - 41 | 240 | 115 | 395
* TN 10m (15 GB, 32 cores) - 430 | 2030 | 850 | 3310

In [10]:
# Start timer
start_time = time.time()

# Crop elevation into tiles
geotiled.crop_into_tiles(input_file='elevation', output_folder='elevation_tiles', num_tiles=4)
crop_time = time.time()
print("Tile split time: %s seconds" % (crop_time - start_time))

# Run GEOtiled
geotiled.compute_geotiled(input_folder='elevation_tiles', param_list=params, use_saga=True, clean_saga=True, cleanup=True)
compute_time = time.time()
print("Tile compute time: %s seconds" % (compute_time - crop_time))

# Merge tiles
geotiled.build_mosaic_filtered(input_folder='slope_tiles', output_file='slope4', cleanup=True)
print("Tile merge time: %s seconds" % (time.time() - compute_time))

# Output total computation time
print("Compute time for 4 tiles: %s seconds" % (time.time() - start_time))

tile_0000.tif cropped.
tile_0001.tif cropped.
tile_0002.tif cropped.
tile_0003.tif cropped.
Tile split time: 430.21942138671875 seconds
Getting input files...
Computing parameters using SAGA...
Converting tile_0000.tif to SDAT...
Converting tile_0001.tif to SDAT...
Converting tile_0002.tif to SDAT...
Converting tile_0003.tif to SDAT...
Converting SAGA files to GeoTIFF...
Cleaning SAGA files
Cleaning files...
GEOtiled computation done!
Tile compute time: 2029.76877450943 seconds
Mosaicking process started...
Building VRT...
Averaging buffer values...
Cleaning files...
Mosaic process completed.
Tile merge time: 850.1826305389404 seconds
Compute time for 4 tiles: 3310.1709179878235 seconds


## 9 Tile Run

In [ ]:
# Start timer
start_time = time.time()

# Crop elevation into tiles
geotiled.crop_into_tiles(input_file='elevation', output_folder='elevation_tiles', num_tiles=9)
crop_time = time.time()
print("Tile split time: %s seconds" % (crop_time - start_time))

# Run GEOtiled
geotiled.compute_geotiled(input_folder='elevation_tiles', param_list=params, use_saga=True, clean_saga=True, cleanup=True)
compute_time = time.time()
print("Tile compute time: %s seconds" % (compute_time - crop_time))

# Merge tiles
geotiled.build_mosaic_filtered(input_folder='slope_tiles', output_file='slope9', cleanup=True)
print("Tile merge time: %s seconds" % (time.time() - compute_time))

# Output total computation time
print("Compute time for 9 tiles: %s seconds" % (time.time() - start_time))

## 16 Tile Run

#### Results

* TN 30m (1.7 GB) - 41 | 336 | 161 | 538
* TN 10m (15 GB, 32 cores) - 407 | 2092 | 1540 | 4039

In [11]:
# Start timer
start_time = time.time()

# Crop elevation into tiles
geotiled.crop_into_tiles(input_file='elevation', output_folder='elevation_tiles', num_tiles=16)
crop_time = time.time()
print("Tile split time: %s seconds" % (crop_time - start_time))

# Run GEOtiled
geotiled.compute_geotiled(input_folder='elevation_tiles', param_list=params, use_saga=True, clean_saga=True, cleanup=True)
compute_time = time.time()
print("Tile compute time: %s seconds" % (compute_time - crop_time))

# Merge tiles
geotiled.build_mosaic_filtered(input_folder='slope_tiles', output_file='slope16', cleanup=True)
print("Tile merge time: %s seconds" % (time.time() - compute_time))

# Output total computation time
print("Compute time for 16 tiles: %s seconds" % (time.time() - start_time))

tile_0000.tif cropped.
tile_0001.tif cropped.
tile_0002.tif cropped.
tile_0003.tif cropped.
tile_0004.tif cropped.
tile_0005.tif cropped.
tile_0006.tif cropped.
tile_0007.tif cropped.
tile_0008.tif cropped.
tile_0009.tif cropped.
tile_0010.tif cropped.
tile_0011.tif cropped.
tile_0012.tif cropped.
tile_0013.tif cropped.
tile_0014.tif cropped.
tile_0015.tif cropped.
Tile split time: 407.1943368911743 seconds
Getting input files...
Computing parameters using SAGA...
Converting tile_0000.tif to SDAT...
Converting tile_0001.tif to SDAT...
Converting tile_0002.tif to SDAT...
Converting tile_0003.tif to SDAT...
Converting tile_0004.tif to SDAT...
Converting tile_0005.tif to SDAT...
Converting tile_0006.tif to SDAT...
Converting tile_0007.tif to SDAT...
Converting tile_0008.tif to SDAT...
Converting tile_0009.tif to SDAT...
Converting tile_0010.tif to SDAT...
Converting tile_0011.tif to SDAT...
Converting tile_0012.tif to SDAT...
Converting tile_0013.tif to SDAT...
Converting tile_0014.tif to

## 25 Tile Run

In [ ]:
# Start timer
start_time = time.time()

# Crop elevation into tiles
geotiled.crop_into_tiles(input_file='elevation', output_folder='elevation_tiles', num_tiles=25)
crop_time = time.time()
print("Tile split time: %s seconds" % (crop_time - start_time))

# Run GEOtiled
geotiled.compute_geotiled(input_folder='elevation_tiles', param_list=params, use_saga=True, clean_saga=True, cleanup=True)
compute_time = time.time()
print("Tile compute time: %s seconds" % (compute_time - crop_time))

# Merge tiles
geotiled.build_mosaic_filtered(input_folder='slope_tiles', output_file='slope25', cleanup=True)
print("Tile merge time: %s seconds" % (time.time() - compute_time))

# Output total computation time
print("Compute time for 25 tiles: %s seconds" % (time.time() - start_time))

## 36 Tile Run

In [ ]:
# Start timer
start_time = time.time()

# Crop elevation into tiles
geotiled.crop_into_tiles(input_file='elevation', output_folder='elevation_tiles', num_tiles=36)
crop_time = time.time()
print("Tile split time: %s seconds" % (crop_time - start_time))

# Run GEOtiled
geotiled.compute_geotiled(input_folder='elevation_tiles', param_list=params, use_saga=True, clean_saga=True, cleanup=True)
compute_time = time.time()
print("Tile compute time: %s seconds" % (compute_time - crop_time))

# Merge tiles
geotiled.build_mosaic_filtered(input_folder='slope_tiles', output_file='slope36', cleanup=True)
print("Tile merge time: %s seconds" % (time.time() - compute_time))

# Output total computation time
print("Compute time for 36 tiles: %s seconds" % (time.time() - start_time))

## 49 Tile Run

In [ ]:
# Start timer
start_time = time.time()

# Crop elevation into tiles
geotiled.crop_into_tiles(input_file='elevation', output_folder='elevation_tiles', num_tiles=49)
crop_time = time.time()
print("Tile split time: %s seconds" % (crop_time - start_time))

# Run GEOtiled
geotiled.compute_geotiled(input_folder='elevation_tiles', param_list=params, use_saga=True, clean_saga=True, cleanup=True)
compute_time = time.time()
print("Tile compute time: %s seconds" % (compute_time - crop_time))

# Merge tiles
geotiled.build_mosaic_filtered(input_folder='slope_tiles', output_file='slope49', cleanup=True)
print("Tile merge time: %s seconds" % (time.time() - compute_time))

# Output total computation time
print("Compute time for 49 tiles: %s seconds" % (time.time() - start_time))

## 64 Tile Run

#### Results

* TN 30m (1.7 GB) - 43 | 528 | 402 | 973
* TN 10m (15 GB, 32 cores) - 426 | 2279 | 4210 | 6916

In [12]:
# Start timer
start_time = time.time()

# Crop elevation into tiles
geotiled.crop_into_tiles(input_file='elevation', output_folder='elevation_tiles', num_tiles=64)
crop_time = time.time()
print("Tile split time: %s seconds" % (crop_time - start_time))

# Run GEOtiled
geotiled.compute_geotiled(input_folder='elevation_tiles', param_list=params, use_saga=True, clean_saga=True, cleanup=True)
compute_time = time.time()
print("Tile compute time: %s seconds" % (compute_time - crop_time))

# Merge tiles
geotiled.build_mosaic_filtered(input_folder='slope_tiles', output_file='slope64', cleanup=True)
print("Tile merge time: %s seconds" % (time.time() - compute_time))

# Output total computation time
print("Compute time for 64 tiles: %s seconds" % (time.time() - start_time))

tile_0000.tif cropped.
tile_0001.tif cropped.
tile_0002.tif cropped.
tile_0003.tif cropped.
tile_0004.tif cropped.
tile_0005.tif cropped.
tile_0006.tif cropped.
tile_0007.tif cropped.
tile_0008.tif cropped.
tile_0009.tif cropped.
tile_0010.tif cropped.
tile_0011.tif cropped.
tile_0012.tif cropped.
tile_0013.tif cropped.
tile_0014.tif cropped.
tile_0015.tif cropped.
tile_0016.tif cropped.
tile_0017.tif cropped.
tile_0018.tif cropped.
tile_0019.tif cropped.
tile_0020.tif cropped.
tile_0021.tif cropped.
tile_0022.tif cropped.
tile_0023.tif cropped.
tile_0024.tif cropped.
tile_0025.tif cropped.
tile_0026.tif cropped.
tile_0027.tif cropped.
tile_0028.tif cropped.
tile_0029.tif cropped.
tile_0030.tif cropped.
tile_0031.tif cropped.
tile_0032.tif cropped.
tile_0033.tif cropped.
tile_0034.tif cropped.
tile_0035.tif cropped.
tile_0036.tif cropped.
tile_0037.tif cropped.
tile_0038.tif cropped.
tile_0039.tif cropped.
tile_0040.tif cropped.
tile_0041.tif cropped.
tile_0042.tif cropped.
tile_0043.t

## 81 Tile Run

In [ ]:
# Start timer
start_time = time.time()

# Crop elevation into tiles
geotiled.crop_into_tiles(input_file='elevation', output_folder='elevation_tiles', num_tiles=81)
crop_time = time.time()
print("Tile split time: %s seconds" % (crop_time - start_time))

# Run GEOtiled
geotiled.compute_geotiled(input_folder='elevation_tiles', param_list=params, use_saga=True, clean_saga=True, cleanup=True)
compute_time = time.time()
print("Tile compute time: %s seconds" % (compute_time - crop_time))

# Merge tiles
geotiled.build_mosaic_filtered(input_folder='slope_tiles', output_file='slope81', cleanup=True)
print("Tile merge time: %s seconds" % (time.time() - compute_time))

# Output total computation time
print("Compute time for 81 tiles: %s seconds" % (time.time() - start_time))

## 100 Tile Run

In [ ]:
# Start timer
start_time = time.time()

# Crop elevation into tiles
geotiled.crop_into_tiles(input_file='elevation', output_folder='elevation_tiles', num_tiles=100)
crop_time = time.time()
print("Tile split time: %s seconds" % (crop_time - start_time))

# Run GEOtiled
geotiled.compute_geotiled(input_folder='elevation_tiles', param_list=params, use_saga=True, clean_saga=True, cleanup=True)
compute_time = time.time()
print("Tile compute time: %s seconds" % (compute_time - crop_time))

# Merge tiles
geotiled.build_mosaic_filtered(input_folder='slope_tiles', output_file='slope100', cleanup=True)
print("Tile merge time: %s seconds" % (time.time() - compute_time))

# Output total computation time
print("Compute time for 100 tiles: %s seconds" % (time.time() - start_time))

## Special Notes

* Data in SAGA format is on average about **double** the size of data in GeoTIFF format

## End of Notebook